In [0]:
pip install category_encoders

In [1]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import SGD
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd
import category_encoders as ce
import os
import pickle
import gc
from tqdm import tqdm
import pickle
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor, GradientBoostingRegressor
from sklearn import ensemble
import xgboost as xgb

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def encode_text_features(encode_decode, data_frame, encoder_isa=None, encoder_mem_type=None):
    # Implement Categorical OneHot encoding for ISA and mem-type
    if encode_decode == 'encode':
        encoder_isa = ce.one_hot.OneHotEncoder(cols=['isa'])
        encoder_mem_type = ce.one_hot.OneHotEncoder(cols=['mem-type'])
        encoder_isa.fit(data_frame, verbose=1)
        df_new1 = encoder_isa.transform(data_frame)
        encoder_mem_type.fit(df_new1, verbose=1)
        df_new = encoder_mem_type.transform(df_new1)
        encoded_data_frame = df_new
    else:
        df_new1 = encoder_isa.transform(data_frame)
        df_new = encoder_mem_type.transform(df_new1)
        encoded_data_frame = df_new
        
    return encoded_data_frame, encoder_isa, encoder_mem_type

In [0]:
def absolute_percentage_error(Y_test, Y_pred):
    error = 0
    for i in range(len(Y_test)):
        if(Y_test[i]!= 0 ):
            error = error + (abs(Y_test[i] - Y_pred[i]))/Y_test[i]
        
    error = error/ len(Y_test)
    return error[0]

In [0]:
def return_best_param(model, grid, X_train, Y_train):
    grid = GridSearchCV(model, grid, refit = True, verbose = 0)
    # fitting the model for grid search 
    tqdm(grid.fit(X_train, Y_train)) 
    print('Found Best Parameters for this model', model)

    # print how our model looks after hyper-parameter tuning 
    return (grid.best_estimator_) 

# Dataset 1 :SHA

In [0]:
def process_all_sha(dataset_path, dataset_name,dataset_name_n,path_for_saving_data):
    
    ################## Data Preprocessing ######################
    
    df = pd.read_csv(dataset_path + dataset_name + '.csv')
    dfn = pd.read_csv(dataset_path + dataset_name_n + '.csv')
    encoded_data_frame, encoder_isa, encoder_mem_type = encode_text_features('encode', df
                                                                             , encoder_isa = None, encoder_mem_type=None)
    encoded_data_frame_n, encoder_isa_n, encoder_mem_type_n = encode_text_features('encode', dfn
                                                                                   , encoder_isa = None, encoder_mem_type=None)
    
    total_data_n = encoded_data_frame_n.drop(columns = ['arch','bus_speed','mem-type_1','mem-type_2','isa_1'])
    total_data = encoded_data_frame.drop(columns = ['arch','mem-type_1','mem-type_2','mem-type_3','mem-type_4','isa_1', 'isa_2'])
    print(total_data.columns, total_data_n.columns, len(total_data.columns), len(total_data_n.columns))
    total_data = total_data.fillna(0)
    total_data_n = total_data_n.fillna(0)
 
    X_sim = total_data.drop(columns = ['runtime']).to_numpy()
    Y_sim = total_data['runtime'].to_numpy()
    X_phy = total_data_n.drop(columns = ['runtime']).to_numpy()
    Y_phy = total_data_n['runtime'].to_numpy()  
    
    print(X_sim.shape, X_phy.shape, Y_sim.shape, Y_phy.shape)

    # Separating Physical data to 10% and 90%
    X_train_phy, X_test_phy, Y_train_phy, Y_test_phy = train_test_split(X_phy, Y_phy, test_size = 0.90, random_state = 0)
    print(X_train_phy.shape, X_test_phy.shape, Y_train_phy.shape, Y_test_phy.shape)
    X_train_sim = np.append(X_sim, X_train_phy,axis = 0)
    Y_train_sim = np.append(Y_sim, Y_train_phy,axis = 0)
    print(X_train_sim.shape, Y_train_sim.shape, X_test_phy.shape, Y_test_phy.shape)
    
    X_train = X_train_sim
    X_test = X_test_phy
    Y_train = Y_train_sim
    Y_test = Y_test_phy
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    X_train = scaler_x.fit_transform(X_train)
    X_test = scaler_x.fit_transform(X_test)
    Y_train = np.reshape(Y_train, (len(Y_train),1))
    Y_test = np.reshape(Y_test, (len(Y_test),1))
    Y_train = scaler_y.fit_transform(Y_train)
    Y_test = scaler_y.fit_transform(Y_test)    
    
    ################## Data Preprocessing ######################
    
    # Put best models here using grid search
    
    # 1. SVR 
    param_grid_svr = {'C': [0.1, 1, 10, 100, 1000],  
              # Regularization parameter. The strength of the regularization is inversely proportional to C. 
              # Must be strictly positive. The penalty is a squared l2 penalty.
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              # Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
              # if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
              # if ‘auto’, uses 1 / n_features.
              
              'kernel': ['rbf']} 
    model_svr = SVR()          
    best_svr = return_best_param(model_svr, param_grid_svr, X_train, Y_train)
    

    
    # 2. LR
    param_grid_lr =  {'fit_intercept': [True, False],  
             'normalize' : [True, False], 
             } 
    model_lr = LinearRegression()          
    best_lr = return_best_param(model_lr, param_grid_lr, X_train, Y_train)    
    
    # 3. RR
    param_grid_rr =   {'alpha': [0.1, 1, 10, 100, 1000],  
             'fit_intercept' : [True, False],
              'normalize' :[True, False],
              'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
             } 
    model_rr = linear_model.Ridge()          
    best_rr = return_best_param(model_rr, param_grid_rr, X_train, Y_train)
    
    # 4. KNN
    param_grid_knn =   {'n_neighbors': [2, 3, 4, 5, 6, 7, 9, 10, 13, 15],  
             'weights' : ['uniform', 'distance'],
              'p' : [1, 2, 4, 5, 7 ,10]
             } 
    model_knn = KNeighborsRegressor()          
    best_knn = return_best_param(model_knn, param_grid_knn, X_train, Y_train) 
    
    # 5. GPR
    param_grid_gpr =   {'alpha': [1e-10, 1e-9, 1e-5, 1e-2],  
             'normalize_y' : [True, False],
                         } 
    model_gpr = GaussianProcessRegressor()          
    best_gpr = return_best_param(model_gpr, param_grid_gpr, X_train, Y_train) 
    # 6. Decision Tree
    param_grid_dt =  {'criterion': ['mse','friedman_mse', 'mae'],  
              'splitter' : ['best', 'random'], 
              'max_depth': [2,3,4,5,7,9,10,15,20,30 ],
              'min_samples_leaf' : [1, 2,3,  4, 5,6, 7],
              'max_features' : ['auto', 'sqrt', 'log2'],
             } 
    model_dt = DecisionTreeRegressor()          
    best_dt = DecisionTreeRegressor(criterion='friedman_mse', max_depth=30,
                      max_features='log2', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=2, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort=False,
                      random_state=None, splitter='best')
    # return_best_param(model_dt, param_grid_dt, X_train, Y_train) 

    # 7. Random Forest 
    param_grid_rf =   {'n_estimators' : [10, 50, 100, 200],
              'criterion': ['mse','friedman_mse', 'mae'],  
              'max_depth': [3,4,5,7,10,15, None ],
              'warm_start': ['True', 'False'],

             } 
    model_rf = RandomForestRegressor()          
    best_rf = RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start='False')
    # return_best_param(model_rf, param_grid_rf, X_train, Y_train) 
    
    # 8. Extra Trees Regressor
    param_grid_etr =   {'n_estimators' : [10, 50, 100, 200],
              'criterion': ['mse','friedman_mse', 'mae'],  
              'max_depth': [3,4,5,7,10,15, None ],
              'warm_start': ['True', 'False']
             }
    model_etr = ExtraTreesRegressor()          
    best_etr =  ExtraTreesRegressor(bootstrap=False, criterion='friedman_mse', max_depth=3,
                    max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
                    oob_score=False, random_state=42, verbose=0,
                    warm_start='True')
    # return_best_param(model_etr, param_grid_etr, X_train, Y_train) 
    
    # 9. GBR
    param_grid_gbr =   {'n_estimators' : [10, 20, 50, 100],
              
              'criterion': ['mse','friedman_mse', 'mae'],  
              'max_depth': [3,4,5,7,10,15, None ],
              'loss' : ['ls', 'lad', 'huber', 'quantile'],
             }
    model_gbr = GradientBoostingRegressor()          
    best_gbr = ensemble.GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
                          learning_rate=0.1, loss='lad', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=50,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
    # return_best_param(model_gbr, param_grid_gbr, X_train, Y_train)     
    
    # 10. XGB
    param_grid_xgb =  {'n_estimators' : [10, 20, 50, 100], 
              'max_depth': [3,4,5,7,10,15],
              'learning_rate' : [0.1, 0.5, 0.01, 0.001, 0.0001]      
             } 
    model_xgb = xgb.XGBRegressor()
    best_xgb = xgb.XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, validate_parameters=False, verbosity=1) 
    # return_best_param(model_xgb, param_grid_xgb, X_train, Y_train)
    
    best_models = [best_svr, best_lr, best_rr, best_knn, best_gpr, best_dt, best_rf, best_etr, best_gbr, best_xgb]
    best_models_name = ['best_svr', 'best_lr', 'best_rr', 'best_knn', 'best_gpr', 'best_dt', 'best_rf', 'best_etr'
                        , 'best_gbr', 'best_xgb']
    k = 0
    
    df = pd.DataFrame(columns = ['model_name', 'dataset_name', 'r2', 'mape'])
    
    for model in best_models:
        model_orig = model
        print('Running model number:', k+1, 'with Model Name: ', best_models_name[k])
        r2_scores = []
        mape_scores = []

        fold = 1
        print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
        model_orig.fit(X_train, Y_train)
        Y_pred_fold = model_orig.predict(X_test)
        Y_test_fold = scaler_y.inverse_transform(Y_test)
        Y_pred_fold = scaler_y.inverse_transform(Y_pred_fold)
        
        # print('Accuracy =',accuracy_score(Y_test, Y_pred))
        r2_scores.append(r2_score(Y_test_fold, Y_pred_fold))
        mape_scores.append(absolute_percentage_error(Y_test_fold, Y_pred_fold))
        
        df = df.append({'model_name': best_models_name[k], 'dataset_name': dataset_name
                        , 'r2': r2_scores[0], 'mape': mape_scores[0],}, ignore_index=True)
        k = k + 1  
    print(df.head())
    df.to_csv('result_transfer_Sim_to_Phy_sha_10' + '.csv')

In [27]:
dataset_name_n = 'sha_physical'
dataset_name = 'sha_simulated'
dataset_path = '/content/drive/My Drive/all_datasets/'
path_for_saving_data = dataset_name
process_all_sha(dataset_path, dataset_name, dataset_name_n, path_for_saving_data)

Index(['cpu-clock', 'l1d_assoc', 'l1d_cache_lines', 'l1d_shared_by_threads',
       'l1d_size', 'l2_assoc', 'l2_cache_lines', 'l2_shared_by_threads',
       'l2_size', 'l3_assoc', 'l3_cache_lines', 'l3_shared_by_threads',
       'l3_size', 'mem-size', 'mem_clock', 'num-cpus', 'runtime'],
      dtype='object') Index(['cpu-clock', 'l1d_assoc', 'l1d_cache_lines', 'l1d_shared_by_threads',
       'l1d_size', 'l2_assoc', 'l2_cache_lines', 'l2_shared_by_threads',
       'l2_size', 'l3_assoc', 'l3_cache_lines', 'l3_shared_by_threads',
       'l3_size', 'mem-size', 'mem_clock', 'num-cpus', 'runtime'],
      dtype='object') 17 17
(475, 16) (52, 16) (475,) (52,)
(5, 16) (47, 16) (5,) (47,)
(480, 16) (480,) (47, 16) (47,)
(480, 16) (47, 16) (480,) (47,)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Found Best Parameters for this model SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Found Best Parameters for this model LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)


0it [00:00, ?it/s]


Found Best Parameters for this model Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)


...

Found Best Parameters for this model KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')


0it [00:00, ?it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:194: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Found Best Parameters for this model GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=None,
                         n_restarts_optimizer=0, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=None)
Running model number: 1 with Model Name:  best_svr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 2 with Model Name:  best_lr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 3 with Model Name:  best_rr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 4 with Model Name:  best_knn
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 5 with Model Name:  best_gpr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 6 with Model Name:  best_dt
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 7 with Model Name:  best_rf
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 8 with Model Name:  best_etr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 9 with Model Name:  best_gbr
(480, 16) (

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:194: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)


Running model number: 10 with Model Name:  best_xgb
(480, 16) (47, 16) (480, 1) (47, 1)
[09:03:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  model_name   dataset_name        r2      mape
0   best_svr  sha_simulated  0.138841  0.139812
1    best_lr  sha_simulated -0.740383  0.215772
2    best_rr  sha_simulated  0.003595  0.156975
3   best_knn  sha_simulated  0.290695  0.134173
4   best_gpr  sha_simulated  0.071567  0.170846


# Dataset 2: Dijkstra

In [0]:
def process_all_dijkstra(dataset_path, dataset_name,dataset_name_n,path_for_saving_data):
    
    ################## Data Preprocessing ######################
    
    df = pd.read_csv(dataset_path + dataset_name + '.csv')
    dfn = pd.read_csv(dataset_path + dataset_name_n + '.csv')
    encoded_data_frame, encoder_isa, encoder_mem_type = encode_text_features('encode', df
                                                                             , encoder_isa = None, encoder_mem_type=None)
    encoded_data_frame_n, encoder_isa_n, encoder_mem_type_n = encode_text_features('encode', dfn
                                                                                   , encoder_isa = None, encoder_mem_type=None)
    
    total_data_n = encoded_data_frame_n.drop(columns = ['arch','bus_speed','mem-type_1','mem-type_2','isa_1'])
    total_data = encoded_data_frame.drop(columns = ['arch','mem-type_1','mem-type_2','mem-type_3','mem-type_4','isa_1', 'isa_2'])
    print(total_data.columns, total_data_n.columns, len(total_data.columns), len(total_data_n.columns))
    total_data = total_data.fillna(0)
    total_data_n = total_data_n.fillna(0)
 
    X_sim = total_data.drop(columns = ['runtime']).to_numpy()
    Y_sim = total_data['runtime'].to_numpy()
    X_phy = total_data_n.drop(columns = ['runtime']).to_numpy()
    Y_phy = total_data_n['runtime'].to_numpy()  
    
    print(X_sim.shape, X_phy.shape, Y_sim.shape, Y_phy.shape)

    # Separating Physical data to 10% and 90%
    X_train_phy, X_test_phy, Y_train_phy, Y_test_phy = train_test_split(X_phy, Y_phy, test_size = 0.90, random_state = 0)
    print(X_train_phy.shape, X_test_phy.shape, Y_train_phy.shape, Y_test_phy.shape)
    X_train_sim = np.append(X_sim, X_train_phy,axis = 0)
    Y_train_sim = np.append(Y_sim, Y_train_phy,axis = 0)
    print(X_train_sim.shape, Y_train_sim.shape, X_test_phy.shape, Y_test_phy.shape)
    
    X_train = X_train_sim
    X_test = X_test_phy
    Y_train = Y_train_sim
    Y_test = Y_test_phy
    print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
    
    scaler_x = StandardScaler()
    scaler_y = StandardScaler()
    X_train = scaler_x.fit_transform(X_train)
    X_test = scaler_x.fit_transform(X_test)
    Y_train = np.reshape(Y_train, (len(Y_train),1))
    Y_test = np.reshape(Y_test, (len(Y_test),1))
    Y_train = scaler_y.fit_transform(Y_train)
    Y_test = scaler_y.fit_transform(Y_test)    
    
    ################## Data Preprocessing ######################
    
    # Put best models here using grid search
    
    # 1. SVR 
    param_grid_svr = {'C': [0.1, 1, 10, 100, 1000],  
              # Regularization parameter. The strength of the regularization is inversely proportional to C. 
              # Must be strictly positive. The penalty is a squared l2 penalty.
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              # Kernel coefficient for ‘rbf’, ‘poly’ and ‘sigmoid’.
              # if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,
              # if ‘auto’, uses 1 / n_features.
              
              'kernel': ['rbf']} 
    model_svr = SVR()          
    best_svr = return_best_param(model_svr, param_grid_svr, X_train, Y_train)
    

    
    # 2. LR
    param_grid_lr =  {'fit_intercept': [True, False],  
             'normalize' : [True, False], 
             } 
    model_lr = LinearRegression()          
    best_lr = return_best_param(model_lr, param_grid_lr, X_train, Y_train)    
    
    # 3. RR
    param_grid_rr =   {'alpha': [0.1, 1, 10, 100, 1000],  
             'fit_intercept' : [True, False],
              'normalize' :[True, False],
              'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
             } 
    model_rr = linear_model.Ridge()          
    best_rr = return_best_param(model_rr, param_grid_rr, X_train, Y_train)
    
    # 4. KNN
    param_grid_knn =   {'n_neighbors': [2, 3, 4, 5, 6, 7, 9, 10, 13, 15],  
             'weights' : ['uniform', 'distance'],
              'p' : [1, 2, 4, 5, 7 ,10]
             } 
    model_knn = KNeighborsRegressor()          
    best_knn = return_best_param(model_knn, param_grid_knn, X_train, Y_train) 
    
    # 5. GPR
    param_grid_gpr =   {'alpha': [1e-10, 1e-9, 1e-5, 1e-2],  
             'normalize_y' : [True, False],
                         } 
    model_gpr = GaussianProcessRegressor()          
    best_gpr = return_best_param(model_gpr, param_grid_gpr, X_train, Y_train) 
    # 6. Decision Tree
    param_grid_dt =  {'criterion': ['mse','friedman_mse', 'mae'],  
              'splitter' : ['best', 'random'], 
              'max_depth': [2,3,4,5,7,9,10,15,20,30 ],
              'min_samples_leaf' : [1, 2,3,  4, 5,6, 7],
              'max_features' : ['auto', 'sqrt', 'log2'],
             } 
    model_dt = DecisionTreeRegressor()          
    best_dt = DecisionTreeRegressor(criterion='mae', max_depth=20, max_features='auto',
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=6,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')
    # return_best_param(model_dt, param_grid_dt, X_train, Y_train) 

    # 7. Random Forest 
    param_grid_rf =   {'n_estimators' : [10, 50, 100, 200],
              'criterion': ['mse','friedman_mse', 'mae'],  
              'max_depth': [3,4,5,7,10,15, None ],
              'warm_start': ['True', 'False'],

             } 
    model_rf = RandomForestRegressor()          
    best_rf = RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start='True')
    # return_best_param(model_rf, param_grid_rf, X_train, Y_train) 
    
    # 8. Extra Trees Regressor
    param_grid_etr =   {'n_estimators' : [10, 50, 100, 200],
              'criterion': ['mse','friedman_mse', 'mae'],  
              'max_depth': [3,4,5,7,10,15, None ],
              'warm_start': ['True', 'False']
             }
    model_etr = ExtraTreesRegressor()          
    best_etr =  ExtraTreesRegressor(bootstrap=False, criterion='mae', max_depth=10,
                    max_features='auto', max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
                    oob_score=False, random_state=42, verbose=0,
                    warm_start='True')
    # return_best_param(model_etr, param_grid_etr, X_train, Y_train) 
    
    # 9. GBR
    param_grid_gbr =   {'n_estimators' : [10, 20, 50, 100],
              
              'criterion': ['mse','friedman_mse', 'mae'],  
              'max_depth': [3,4,5,7,10,15, None ],
              'loss' : ['ls', 'lad', 'huber', 'quantile'],
             }
    model_gbr = GradientBoostingRegressor()          
    best_gbr = ensemble.GradientBoostingRegressor(alpha=0.9, criterion='mse', init=None,
                          learning_rate=0.1, loss='lad', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=50,
                          n_iter_no_change=None, presort='auto',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)
    # return_best_param(model_gbr, param_grid_gbr, X_train, Y_train)     
    
    # 10. XGB
    param_grid_xgb =  {'n_estimators' : [10, 20, 50, 100], 
              'max_depth': [3,4,5,7,10,15],
              'learning_rate' : [0.1, 0.5, 0.01, 0.001, 0.0001]      
             } 
    model_xgb = xgb.XGBRegressor()
    best_xgb = xgb.XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, validate_parameters=False, verbosity=1)
    # return_best_param(model_xgb, param_grid_xgb, X_train, Y_train)
    
    best_models = [best_svr, best_lr, best_rr, best_knn, best_gpr, best_dt, best_rf, best_etr, best_gbr, best_xgb]
    best_models_name = ['best_svr', 'best_lr', 'best_rr', 'best_knn', 'best_gpr', 'best_dt', 'best_rf', 'best_etr'
                        , 'best_gbr', 'best_xgb']
    k = 0
    
    df = pd.DataFrame(columns = ['model_name', 'dataset_name', 'r2', 'mape'])
    
    for model in best_models:
        model_orig = model
        print('Running model number:', k+1, 'with Model Name: ', best_models_name[k])
        r2_scores = []
        mape_scores = []

        fold = 1
        print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
        model_orig.fit(X_train, Y_train)
        Y_pred_fold = model_orig.predict(X_test)
        Y_test_fold = scaler_y.inverse_transform(Y_test)
        Y_pred_fold = scaler_y.inverse_transform(Y_pred_fold)
        
        # print('Accuracy =',accuracy_score(Y_test, Y_pred))
        r2_scores.append(r2_score(Y_test_fold, Y_pred_fold))
        mape_scores.append(absolute_percentage_error(Y_test_fold, Y_pred_fold))
        
        df = df.append({'model_name': best_models_name[k], 'dataset_name': dataset_name
                        , 'r2': r2_scores[0], 'mape': mape_scores[0],}, ignore_index=True)
        k = k + 1  
    print(df.head())
    df.to_csv('result_transfer_Sim_to_Phy_dijkstra_10' + '.csv')

In [29]:
dataset_name_n = 'dijkstra_physical'
dataset_name = 'dijkstra_simulated'
dataset_path = '/content/drive/My Drive/all_datasets/'
path_for_saving_data = dataset_name
process_all_dijkstra(dataset_path, dataset_name, dataset_name_n, path_for_saving_data)

Index(['cpu-clock', 'l1d_assoc', 'l1d_cache_lines', 'l1d_shared_by_threads',
       'l1d_size', 'l2_assoc', 'l2_cache_lines', 'l2_shared_by_threads',
       'l2_size', 'l3_assoc', 'l3_cache_lines', 'l3_shared_by_threads',
       'l3_size', 'mem-size', 'mem_clock', 'num-cpus', 'runtime'],
      dtype='object') Index(['cpu-clock', 'l1d_assoc', 'l1d_cache_lines', 'l1d_shared_by_threads',
       'l1d_size', 'l2_assoc', 'l2_cache_lines', 'l2_shared_by_threads',
       'l2_size', 'l3_assoc', 'l3_cache_lines', 'l3_shared_by_threads',
       'l3_size', 'mem-size', 'mem_clock', 'num-cpus', 'runtime'],
      dtype='object') 17 17
(475, 16) (52, 16) (475,) (52,)
(5, 16) (47, 16) (5,) (47,)
(480, 16) (480,) (47, 16) (47,)
(480, 16) (47, 16) (480,) (47,)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

Found Best Parameters for this model SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
Found Best Parameters for this model LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)


0it [00:00, ?it/s]


Found Best Parameters for this model Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)


...

0it [00:00, ?it/s]
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/tree/_classes.py:319: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:193: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Found Best Parameters for this model GaussianProcessRegressor(alpha=1e-10, copy_X_train=True, kernel=None,
                         n_restarts_optimizer=0, normalize_y=False,
                         optimizer='fmin_l_bfgs_b', random_state=None)
Running model number: 1 with Model Name:  best_svr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 2 with Model Name:  best_lr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 3 with Model Name:  best_rr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 4 with Model Name:  best_knn
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 5 with Model Name:  best_gpr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 6 with Model Name:  best_dt
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 7 with Model Name:  best_rf
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 8 with Model Name:  best_etr
(480, 16) (47, 16) (480, 1) (47, 1)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:193: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Running model number: 9 with Model Name:  best_gbr
(480, 16) (47, 16) (480, 1) (47, 1)
Running model number: 10 with Model Name:  best_xgb
(480, 16) (47, 16) (480, 1) (47, 1)
[09:21:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
  model_name        dataset_name        r2      mape
0   best_svr  dijkstra_simulated  0.234433  0.133559
1    best_lr  dijkstra_simulated  0.497401  0.102197
2    best_rr  dijkstra_simulated  0.597931  0.083885
3   best_knn  dijkstra_simulated  0.060386  0.151155
4   best_gpr  dijkstra_simulated  0.061222  0.158275


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1342: FutureWarning: The parameter 'presort' is deprecated and has no effect. It will be removed in v0.24. You can suppress this warning by not passing any value to the 'presort' parameter. We also recommend using HistGradientBoosting models instead.
  FutureWarning)
